# Production ready crosslight detection

### Lets create our model and load the weights

In [ ]:
import display

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

from picamera import PiCamera, array
from time import sleep
from numpy import around

In [ ]:
image_height = 128
image_width = 512

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(image_height, image_width, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# the model so far outputs 3D feature maps (height, width, features)

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(1))
model.add(Dense(6))
model.add(Activation('sigmoid'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.load_weights('/home/pi/repositories/pren/models/ciffer.h5')

### Raspberry camera logic

In [ ]:
def checkPrediction(prediction):
    if prediction[0] >= 0.95:
        display.one()
        print("1")
        break

    if prediction[1] >= 0.95:
        display.two()
        print("2")
        break

    if prediction[2] >= 0.95:
        display.three()
        print("3")
        break

    if prediction[3] >= 0.95:
        display.four()
        print("4")
        break

    if prediction[4] >= 0.95:
        display.five()
        print("5")
        break

In [ ]:
with PiCamera() as camera:
    camera.resolution = (128, 512)
    sleep(1)
    number_found = False
    
    
    display.one()
    with array.PiRGBArray(camera) as output:
        while True:
            camera.capture(output, 'rgb')
            
            # Lösungsweg 1
            img = output.array.astype(float)
            img = img.reshape((1,) + img.shape)
            prediction = around(model.predict(img), 3)[0]

            # Lösungsweg 2
            #prediction = around(model.predict(output.array), 3)[0]
            
            print(prediction)
            checkPrediction(prediction)
            
            output.truncate(0)